# Setup Variables

In [1]:
REPO_PATH = '/home/ah19/runtime-monitoring'
DATASET = 'MNIST'
PREFIX = 'Regularizations'
FILENAME_POSTFIX = f"{DATASET}_{PREFIX}"
SEED = 42

CUDA = 1
GPU_NAME = f'cuda:{CUDA}'
model_setup = {'batchnorm': True, 'first_layer_norm': False}
model_config = {'lr': 0.01,'epochs': 5}

# Libraries

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from fastprogress import progress_bar
from pathlib import Path
from time import perf_counter
from itertools import product

from sklearn.model_selection import train_test_split
import torch
from torch.backends import cudnn
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary

In [3]:
import sys
sys.path.append(REPO_PATH + '/utilities')
sys.path.append(f'{REPO_PATH}/{DATASET}/trainingModels')

In [6]:
from utils import *
from pathManager import fetchPaths
from MNIST_Model import MNIST_Model

model_ = MNIST_Model

LIMIT_TRAIN_DATA = 0.2
LIMIT_TEST_DATA = 0.05

# GPU Device & Seed

In [7]:
# np.random.seed(SEED)
# torch.manual_seed(SEED)
# torch.cuda.manual_seed(SEED)

In [8]:
device = get_device(GPU_NAME)
torch.cuda.get_device_name(device)

'Tesla T4'

# Paths

In [9]:
base = Path(REPO_PATH)
paths = fetchPaths(base, DATASET)

path = paths[DATASET.lower()]
path_dataset = paths['dataset']
path_trainingModels = paths['trainingModels']
path_trainingModels_regularization = paths['trainingModels_regularization']

path

PosixPath('/home/ah19/runtime-monitoring/MNIST')

# Load / Split / DataLoader

In [10]:
feature_names = get_labels(DATASET)

tf_train = T.Compose([
    T.ToTensor(),
    T.Normalize((0.1307), (0.3015))
])


tf_test = T.Compose([
    T.ToTensor(),
    T.Normalize((0.1307), (0.3015))
])

train_data = get_dataset(DATASET, path_dataset, train=True, transform=tf_train)
test_data = get_dataset(DATASET, path_dataset, train=False, transform=tf_test)

train_split = split_data(train_data.targets, LIMIT_TRAIN_DATA)
test_split = split_data(test_data.targets, LIMIT_TEST_DATA)


train_data = torch.utils.data.Subset(train_data, train_split)
test_data = torch.utils.data.Subset(test_data, test_split)

len(train_data), len(test_data)

(12000, 500)

# Model

In [11]:
def reg_type(l2, l1, p):
    name = ''
    if l2 > 0 and l1 == 0: name += 'L2'
    elif l2 == 0 and l1 > 0: name += 'L1'
    elif l2 > 0 and l1 > 0: name += 'Elastic'
    
    if p > 0 and name == '': name = 'Dropout'
    elif p > 0: name += '-Dropout'
        
    if name == '': name = 'No Regularization'
        
    return name

In [12]:
L2 = [0, 1e-3, 1e-2]
L1 = [0, 1e-3, 1e-2]
dropout = [0.0, 0.3]
optims = ['Adam', 'SGD']
batchsizes = [32, 64, 128, 256]

In [13]:
df_scores = pd.DataFrame({
    'optim': [],
    'batchsize': [],
    'lr': [],
    'regularization': [],
    'L2': [],
    'L1': [],
    'dropout': [],
    'test_accuracy': [],
    'test_loss': [],
    'train_accuracy': [],
    'train_loss': [],
    'time': [],
})


mp = progress_bar( range(len(optims)*len(batchsizes)*len(L2)*len(L1)*len(dropout)) )
for _, (optim, batchsize, l2_, l1_, p) in zip(mp, product(optims, batchsizes, L2, L1, dropout)):
    
    start_time = perf_counter()
    
    model_setup['dropout'] = p
    model_config['batch_size'] = batchsize
    model_config['L2'] = l2_
    model_config['L1'] = l1_

    trainloader = get_dataLoader(train_data, batchsize, True)
    testloader = get_dataLoader(test_data, batchsize, False)

    print('['+'='*50+']')
    print(f"{optim} - {batchsize}: ({l2_}, {l1_}, {p})")

    model = model_(**model_setup).to(device)
    
    nn.DataParallel(model, device_ids=[CUDA])
    cudnn.benchmark = True

    loss_function = nn.CrossEntropyLoss()
    optimizer = getattr(torch.optim, optim)(model.parameters(), lr=model_config['lr'])


    kwargs = {
        'model': model,
        'loss_function': loss_function,
        'optimizer': optimizer,
        'lr_scheduler': None,
        'map_classes': None,
        'skip_classes': None,
        'device': device,
        'model_path': None,
        'trainloader': trainloader,
        'testloader': testloader,
        'config': model_config
    }

    train_losses, test_losses, train_accs, test_accs, test_loss, test_acc, confusion_matrix_test, best_model_name = run_training_testing(**kwargs)
    
    end_time = round(perf_counter() - start_time, 3)
    
    df_scores.loc[df_scores.shape[0] + 1] = [
        optim, batchsize, model_config['lr'], reg_type(l2_, l1_, p), l2_, l1_, p, test_accs[-1], test_losses[-1], train_accs[-1], train_losses[-1], end_time
    ]

[==================================================]
Adam - 32: (0, 0, 0.0)


[==================================================]
Adam - 32: (0, 0, 0.3)


[==================================================]
Adam - 32: (0, 0.001, 0.0)


[==================================================]
Adam - 32: (0, 0.001, 0.3)


[==================================================]
Adam - 32: (0, 0.01, 0.0)


...

[==================================================]
SGD - 256: (0.01, 0, 0.3)


[==================================================]
SGD - 256: (0.01, 0.001, 0.0)


[==================================================]
SGD - 256: (0.01, 0.001, 0.3)


[==================================================]
SGD - 256: (0.01, 0.01, 0.0)


[==================================================]
SGD - 256: (0.01, 0.01, 0.3)


In [16]:
df_scores = df_scores.sort_values(
    ['test_accuracy','test_loss','train_accuracy','train_loss'],
    ascending=[False, True, False, True]
).reset_index(drop=True)

df_scores.to_csv(path_trainingModels_regularization / f'scores-{DATASET.lower()}-{PREFIX.lower()}.csv', index=False)

In [17]:
df_scores.head(7)

,optim,batchsize,lr,regularization,L2,L1,dropout,test_accuracy,test_loss,train_accuracy,train_loss,time
0,Adam,128.0,0.01,No Regularization,0.000,0.000,0.0,0.974,0.065890,0.996917,0.012617,19.821
1,Adam,32.0,0.01,Dropout,0.000,0.000,0.3,0.974,0.091731,0.940167,0.186390,36.278
2,Adam,32.0,0.01,No Regularization,0.000,0.000,0.0,0.972,0.071462,0.985333,0.045915,49.117
3,Adam,64.0,0.01,Dropout,0.000,0.000,0.3,0.972,0.073828,0.947917,0.160027,18.433
4,SGD,32.0,0.01,Elastic,0.001,0.001,0.0,0.970,0.121799,0.979000,0.124932,29.548
5,SGD,32.0,0.01,L2,0.001,0.000,0.0,0.970,0.144889,0.976083,0.129819,29.395
6,Adam,256.0,0.01,Dropout,0.000,0.000,0.3,0.968,0.086753,0.952583,0.149193,18.716
